# Folium, Mapbox, and Shapely
* *Folium*: interactive map visualisation
* *Mapbox*: geocoding and isochrone api
* *Shapely*: calculating polygon's areas and intersections

In [1]:
import pandas as pd
import numpy as np

import requests
import json

import folium

import geopandas as gpd
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon

In [2]:
def get_coordinates(address):
    '''
    Geocoding function returning (lat,lon)
    Argument(s):
    -- address (str)
    Output:
    -- tuple/numeric
    '''
    
    # get request
    req = requests.get("https://api.mapbox.com/geocoding/v5/mapbox.places/"+
                           address.replace(" ","%20")+
                           ".json?access_token=pk.eyJ1IjoicGFsYWRpbmljIiwiYSI6ImNrbTM2cmc3bTMwY3cydm13ZXc2NGVqanIifQ.TmX_nwi8T0k6xCTfjhfvcQ&cachebuster=1615406248283&"+
                           "autocomplete=true")
    
    # get content
    content = req.content
    # load as json
    content = json.loads(content)
    
    # exract lon,lat
    lon,lat = content["features"][0]["center"]
    
    return lat, lon


def get_isochrone(lat,lon,mins,v="driving"):
    '''
    Single Isochrone function returning polygon as list
    Argument(s):
    -- lat (numeric)
    -- lon (numeric)
    -- min (numeric)
    -- v (string) one of ["walking","cycling","driving"]
    Output:
    -- list (list/numeric)
    '''
    # get request
    req = requests.get("https://api.mapbox.com/isochrone/v1/mapbox/"+
                        v+"/"+str(lon)+"%2C"+str(lat)+
                       "?contours_minutes="+str(mins)+
                       "&contours_colors="+"54278f"+
                       "&polygons=true&generalize=0&"+
                       "access_token="+"pk.eyJ1IjoicGFsYWRpbmljIiwiYSI6ImNrbTM2cmc3bTMwY3cydm13ZXc2NGVqanIifQ.TmX_nwi8T0k6xCTfjhfvcQ")

    # get content
    content = req.content
    # load as json
    content = json.loads(content)

    # get coords
    shp = content['features'][0]['geometry']['coordinates']
    # flip lat lon
    shp = np.flip(np.array(shp[0]),axis=1)
    
    return shp


def get_isochrones(lat,lon,mins,v="driving"):
    '''
    Multiple Isochrones function returning polygons as list
    Argument(s):
    -- lat (numeric)
    -- lon (numeric)
    -- min (numeric)
    -- v (string) one of ["walking","cycling","driving"]
    Output:
    -- list (list/numeric)
    '''
    shp = []
    mins.sort(reverse=1)

    for m in mins:
        shp.append(get_isochrone(lat,lon,m,v))
        
    return shp

In [3]:
address= "via spadolini 10, milano"
mins = [5,15,10,20,30,40,50,60]
cols = ["black","pink","purple","red","orange","yellow","green","blue"]
means = "walking"

# Geocoding: address to coordinates

In [4]:
lat,lon = get_coordinates(address)
lat2,lon2 = get_coordinates("via valtellina 40, milano")

# Isochrones

In [5]:
# get isochrone polygon
shp = get_isochrones(lat, lon, mins,v=means)
shp2 = get_isochrones(lat2, lon2, mins,v=means)

In [6]:
# create map
mp = folium.Map([lat,lon],zoom_start=12)

# add isochrone polygon
for i in range(len(shp)):
    folium.Polygon(shp[i],
                   tooltip=mins[i],
                   label = mins[i],
                   fill_opacity=.3,
                   fill_color=cols[i],
                   stroke=0).add_to(mp)

# add isochrone polygon
for i in range(len(shp2)):
    folium.Polygon(shp2[i],
                   tooltip=mins[i],
                   label = mins[i],
                   fill_opacity=0.3,
                   fill_color=cols[i],
                   stroke=0).add_to(mp)
    

# add location marker
folium.Marker([lat,lon],tooltip=address).add_to(mp)

# show map
mp

# Point in Polygon

In [7]:
point = Point(lat, lon)
polygon = Polygon(shp[0])
print(polygon.contains(point))

True


# Polygon Overlap

In [8]:
p0 = Polygon(shp[0])
p1 = Polygon(shp[1])
p2 = Polygon(shp[2])
p3 = Polygon(shp[3])
p4 = Polygon(shp[4])
p5 = Polygon(shp[5])
p6 = Polygon(shp[6])
p7 = Polygon(shp[7])
p8 = Polygon(shp2[0])
p9 = Polygon(shp2[1])
p10 = Polygon(shp2[2])
p11 = Polygon(shp2[3])
p12 = Polygon(shp2[4])
p13 = Polygon(shp2[5])
p14 = Polygon(shp2[6])
p15 = Polygon(shp2[7])
overlap = p0.intersection(p1)
print('Coverage: ', round(overlap.area/p0.area*100,2),'%')

Coverage:  71.75 %


# Merge Polygons

In [25]:
newdata = gpd.GeoDataFrame()
newdata['geometry'] = None

newdata.loc[0, 'geometry'] = p0
newdata.loc[1, 'geometry'] = p1
newdata.loc[2, 'geometry'] = p2
newdata.loc[3, 'geometry'] = p3
newdata.loc[4, 'geometry'] = p4
newdata.loc[5, 'geometry'] = p5
newdata.loc[6, 'geometry'] = p6
newdata.loc[7, 'geometry'] = p7
newdata.loc[8, 'geometry'] = p8
newdata.loc[9, 'geometry'] = p9
newdata.loc[10, 'geometry'] = p10
newdata.loc[11, 'geometry'] = p11
newdata.loc[12, 'geometry'] = p12
newdata.loc[13, 'geometry'] = p13
newdata.loc[14, 'geometry'] = p14
newdata.loc[15, 'geometry'] = p15

In [26]:
newdata

,geometry
0,"POLYGON ((45.48168 9.19910, 45.48168 9.19907, ..."
1,"POLYGON ((45.47492 9.20534, 45.47499 9.20507, ..."
2,"POLYGON ((45.46883 9.20025, 45.46892 9.20007, ..."
3,"POLYGON ((45.46206 9.19748, 45.46218 9.19707, ..."
4,"POLYGON ((45.45566 9.19307, 45.45566 9.19307, ..."
5,"POLYGON ((45.45267 9.19409, 45.45267 9.19407, ..."
6,"POLYGON ((45.44908 9.19450, 45.44933 9.19407, ..."
7,"POLYGON ((45.44578 9.19120, 45.44582 9.19107, ..."
8,"POLYGON ((45.53643 9.19270, 45.53645 9.19264, ..."
9,"POLYGON ((45.52872 9.19299, 45.52908 9.19264, ..."


In [28]:
newdata['new_column'] = [0,1,2,3,4,5,6,7,0,1,2,3,4,5,6,7]
newdata

,geometry,new_column
0,"POLYGON ((45.48168 9.19910, 45.48168 9.19907, ...",0
1,"POLYGON ((45.47492 9.20534, 45.47499 9.20507, ...",1
2,"POLYGON ((45.46883 9.20025, 45.46892 9.20007, ...",2
3,"POLYGON ((45.46206 9.19748, 45.46218 9.19707, ...",3
4,"POLYGON ((45.45566 9.19307, 45.45566 9.19307, ...",4
5,"POLYGON ((45.45267 9.19409, 45.45267 9.19407, ...",5
6,"POLYGON ((45.44908 9.19450, 45.44933 9.19407, ...",6
7,"POLYGON ((45.44578 9.19120, 45.44582 9.19107, ...",7
8,"POLYGON ((45.53643 9.19270, 45.53645 9.19264, ...",0
9,"POLYGON ((45.52872 9.19299, 45.52908 9.19264, ...",1


In [29]:
newdata = newdata.dissolve(by='new_column')
newdata

,geometry
new_column,
0,"POLYGON ((45.53643 9.19270, 45.53645 9.19264, ..."
1,"POLYGON ((45.52872 9.19299, 45.52908 9.19264, ..."
2,"POLYGON ((45.52144 9.19271, 45.52151 9.19264, ..."
3,"MULTIPOLYGON (((45.51443 9.17370, 45.51449 9.1..."
4,"MULTIPOLYGON (((45.50746 9.17874, 45.50750 9.1..."
5,"MULTIPOLYGON (((45.50370 9.18497, 45.50387 9.1..."
6,"MULTIPOLYGON (((45.50047 9.18474, 45.50046 9.1..."
7,"MULTIPOLYGON (((45.49666 9.18493, 45.49697 9.1..."


In [40]:
newdata["key"] = mins
newdata["time"] = mins

In [34]:
newdata.head()

,geometry,mins,type
new_column,,,
0,"POLYGON ((45.53643 9.19270, 45.53645 9.19264, ...",60,walking
1,"POLYGON ((45.52872 9.19299, 45.52908 9.19264, ...",50,walking
2,"POLYGON ((45.52144 9.19271, 45.52151 9.19264, ...",40,walking
3,"MULTIPOLYGON (((45.51443 9.17370, 45.51449 9.1...",30,walking
4,"MULTIPOLYGON (((45.50746 9.17874, 45.50750 9.1...",20,walking


In [13]:
# create map
mp = folium.Map([lat,lon],zoom_start=12)

# for each polygon
for i in range(len(cols)):
    
    # check if multipolygon
    try:
        n = len(newdata.loc[i, 'geometry'])
    except:
        n = 1
        
    if n > 1:
        for j in range(n):
            # add isochrone polygon
            folium.Polygon(np.array(list(newdata.loc[i, 'geometry'][j].exterior.coords)),
                           tooltip=mins[i],
                           label = mins[i],
                           fill_opacity=0.3,
                           fill_color=cols[i],
                           stroke=.1,
                           color="white").add_to(mp)
    else:
        # add isochrone polygon
        folium.Polygon(np.array(list(newdata.loc[i, 'geometry'].exterior.coords)),
                       tooltip=mins[i],
                       label = mins[i],
                       fill_opacity=0.3,
                       fill_color=cols[i],
                       stroke=.1,
                       color="white").add_to(mp)


# add location marker
folium.Marker([lat,lon],tooltip=address).add_to(mp)

# show map
mp

In [52]:
newdata2 = newdata

In [69]:
shp2 = get_isochrones(lon2, lat2, mins,v=means)

In [75]:
newdata2 = gpd.GeoDataFrame()
newdata2['geometry'] = None
newdata2.crs = "EPSG:4326"

p0 = Polygon(shp2[0])
p1 = Polygon(shp2[1])
p2 = Polygon(shp2[2])
p3 = Polygon(shp2[3])
p4 = Polygon(shp2[4])
p5 = Polygon(shp2[5])
p6 = Polygon(shp2[6])
p7 = Polygon(shp2[7])

newdata2.loc[0, 'geometry'] = p0
newdata2.loc[1, 'geometry'] = p1
newdata2.loc[2, 'geometry'] = p2
newdata2.loc[3, 'geometry'] = p3
newdata2.loc[4, 'geometry'] = p4
newdata2.loc[5, 'geometry'] = p5
newdata2.loc[6, 'geometry'] = p6
newdata2.loc[7, 'geometry'] = p7

newdata2["key"] = mins
newdata2["time"] = mins

In [76]:
# Add hover functionality.
style_function = lambda x: {'fillColor': '#ffffff', 
                            'color':'#000000', 
                            'fillOpacity': 0.1, 
                            'weight': 0.1}
highlight_function = lambda x: {'fillColor': '#000000', 
                                'color':'#000000', 
                                'fillOpacity': 0.50, 
                                'weight': 0.1}


mp2 = folium.Map([lat,lon],zoom_start=12)

folium.Choropleth(data=newdata2,
                  geo_data = newdata2,
                  columns=["key","time"],
                  fill_color='BuPu',
                  fill_opacity=0.3,
                  line_opacity=1.0).add_to(mp2)

mp2

In [42]:
NIL = folium.features.GeoJson(
    data = newdata,
    style_function=style_function, 
    control=False,
    highlight_function=highlight_function,
)

mp.add_child(NIL)
mp.keep_in_front(NIL)

In [43]:
mp